## Setup
#### Load the API key and relevant Python libaries.

In [1]:
import io
from dotenv import dotenv_values, load_dotenv, find_dotenv
import openai
from openai import OpenAI
import os
from copy import deepcopy
import json
import time
import ast
import re

import pprint

# LLM Open AI

In [2]:
# Get the first key from the uploaded dictionary
env_file_key = "../../auixiliary/env_GENERAL"

# Open the file and read its content
with open(env_file_key, 'r', encoding='utf-8') as file:
    env_content = file.read()

# Load the content into a variable
env_variables = dotenv_values(stream=io.StringIO(env_content))

api_key = env_variables['OPENAI_API_KEY']
# openai.api_key = api_key

client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

# Models

In [3]:
def chat_gpt(prompt, temperature=0):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=prompt,
        temperature=temperature 
    )
    return response.choices[0].message.content.strip()

# Iterate and Save Use Riskiness Results

# Functions

In [4]:
def replace_key(d, old_key, new_key):
  """
  Replace `old_key` with `new_key` in dictionary `d`.
  The associated value is retained.
  """
  if old_key in d:
      d[new_key] = d.pop(old_key)
  return d

## Read In Prompt Result

In [9]:
def read_prompt_output(file_path):

  # Read the uploaded file
  with open(file_path, 'r') as file:
      data = json.load(file)

  return data

In [10]:
SDG_benefits = read_prompt_output("../../results/SDGs/FULL_SDG_benefits.json")
HR_benefits = read_prompt_output("../../results/HRs/FULL_HR_benefits.json")

In [11]:
HR_benefit_ids = [el["id"] for el in HR_benefits]
SDG_benefit_ids = [el["id"] for el in SDG_benefits]

In [12]:
benefit_ids = set(HR_benefit_ids+SDG_benefit_ids)

# PROMPT: Categorisation

In [19]:
# Assuming you have the variables domain, purpose, aiCapability, aiUser, and aiSubject defined with appropriate values

MESSAGES = [
    {
        'role': 'system',
        'content': """As a distinguished expert in artificial intelligence technology, you embody the forefront of Responsible Artificial Intelligence (RAI). Your expertise is not just technical; it's deeply rooted in a conscientious approach to the ethical, social, and environmental implications of AI. With a wealth of experience, you navigate the intricate balance between harnessing AI's potential for positive impact and mitigating its benefits. Your work is pivotal in aligning AI innovations with the Sustainable Development Goals (SDGs), safeguarding Human Rights (HRs), and ensuring the welfare of individuals, society, and the planet. Your decisiveness, coupled with a profound understanding of AI's capabilities and benefits, positions you as a key influencer in promoting responsible AI practices that respect ethical boundaries and contribute to a sustainable future."""
    },
    {
        'role': 'user',
        'content': """You are provided with an AI technology use description through five concepts:

        UseID: "{}",
        Domain: "{}",
        Purpose: "{}",
        Capability: "{}",
        AI User: "{}",
        AI Subject: "{}"

        followed by a list of benefits of that AI use, including SDG benefits:

        "SDG Assessment":
        "{}"

        and human rights benefits:

        "Assessment of impact on human rights":
        "{}"

        Please be aware that in certain scenarios, there may be an absence of benefits related to either Human Rights (HRs) or Sustainable Development Goals (SDGs). In such instances, you should disregard the missing benefit category and proceed with the evaluation based on the available benefit information, if any. If a scenario arises where no benefit information is provided for a specific use case, pertaining to neither SDGs nor HRs, then you should generate an output in the form of an empty JSON structure, as illustrated below:
        {{
          "Overall benefit Assessment": []
        }}
        ***Check your planned output before outputting it: if it contains any explanations besides the JSON string, omit the explanations. Make sure to output ONLY a correctly formatted JSON string and nothing else.***

        If there are any benefits provided, you will group theses benefits of the given technology use along two axes:
        Axis 1: capability, human interaction, and systemic,
        Axis 2: AI Subject, AI User, and Institutions, General Public and Environment,

        where for the first axis, the three evaluation layers for benefits are distinguished by the target of analysis:

        * Capability: targets AI systems and their technical components. These are routinely evaluated in isolation, including tests of AI system behavior in response to novel tasks or environments, or testing individual technical artefacts, such as the data that an AI system is trained on. It also includes evaluating processes by which these artefacts are created, such as the aggregation mechanisms in processes that are used to adapt an AI system to a particular task.

        * Human Interaction: targets the experience of people interacting with a given AI system. This includes usability: does the AI system perform its intended function at the point of use, and how do experiences differ between user groups? This layer also centres potential externalities: does human–AI interaction lead to unintended effects on the person interacting or exposed to AI outputs? Evaluation at this layer acknowledges that AI system safety depends on who uses an AI system, with what goal in mind, and in what context.

        * Systemic Impact: targets the impact of an AI system on the broader systems in which it is embedded, such as society, the economy, and the natural environment.


        and for the second axis, they are distinguished by the stakeholder:

        * AI User: The entity or individual in charge of deploying and managing the AI system, including individuals, organizations, corporations, public authorities, and agencies responsible for its operation and management

        * AI Subject: The individual directly affected by the use of the AI system, experiencing its effects and consequences. They interact with or are impacted by the AI system's processes, decisions, or outcomes,

        * Institutions, General Public and Environment: includes societal and nature elements affected by the AI system use.


        NB: It is very important that each "Benefit Description" you create be formatted like this: Verb + Object + [Explanation], and is concise, consisting of one clear, to-the-point sentence, with up to maximum of 20 words.
          Specifically, start with an action capability verb in active present tense, where the capability verb examples are given below:
          
                  Capability Verbs and Synonyms:
                      Estimate (Rate, Grade, Measure, Assess)
                      Forecast (Predict, Guess, Speculate)
                      Compare (Rank, Order, Find Best, Find Cheapest, Recommend)
                      Detect (Monitor, Sense, Notice, Classify, Discriminate)
                      Identify (Recognize, Discern, Find, Classify, Perceive)
                      Discover (Extract, Notice, Organize, Cluster, Group, Connect, Reveal)
                      Generate (Make, Compose, Construct, Create, Author)
                      Act (Do, Execute, Play, Go, Learn, Operate, ...) 


          followed by the object and the reason in case it is not obvious and requires an explanation. Since we are talking about potential benefits in general, there is no need to use the word potentially anywhere. Hence, this would NOT be a good output: "Potentially creates jobs in the tech industry and promote innovation in the legal sector." while this is good: "Creates jobs in the tech industry and promotes innovation in the legal sector."
          Also, we want these descriptions to be read by broad public who does not have a deep knowledge about Human Rights or Sustainable Development Goals. Hence, we want descriptions without any mention of specific Human Right Article IDs, or Sustainable Development Goal Targets and Indicators. For instance, these two are NOT good outputs: 1) "Promotes goal 17 by recognizing and highlighting their performances." and 2) "Promotes the enhancement of North-South, South-South and triangular regional and international cooperation on and access to science, technology and innovation.", and these two, instead, are good: 1) "Promotes the social inclusion of all athletes by recognizing and highlighting their performances.", and 2) "Promotes regional and international cooperation on and access to science, technology and innovation."

        Other examples of well-formatted "Benefit Descriptions":
        
        * Reduces the impacts and losses caused by terrorist activities, making cities and human settlements safer.
        * Enhances child safety through early warning and risk reduction.

      Finally, and crucially, examine the benefits you have identified for this specific AI use to ensure their uniqueness. Specifically, if a benefit is initially categorized under both 'Capability' and 'Human Interaction' or any other overlapping categories from the Axis 1, select the most appropriate single category for it. Exclude it from any additional categories. By maintaining a distinct benefit set for each AI use, assign unique Benefit Identifiers (BIDs) accordingly (this should be only numbers). Subsequently, for comprehensive tracking, we will generate a universal Benefit Identifier for all AI uses by incorporating the Use Identifier (UseID) provided in input, in the following structure: "UseID-BID".

      Your output should be in the following format:
      
      {{
        "Overall Benefit Assessment": [
      {{
        "Capability": [
            {{"Benefit ID": "UseID-BID",
             "Benefit Description": "Benefit that targets AI systems, their technical components, and the processes by which these systems and components are created",
             "Stakholders affected by Benefit": ["one or more of AI User, AI Subject, Institutions, General Public and Environment"],
             "SDGs affected by Benefit": ["zero or more SDGs"],
             "Human Rights affected by Benefit": ["zero or more HR articles"]
            }},
            ...
            {{"Benefit ID": "UseID-BID",
             "Benefit Description": "Benefit that targets AI systems, their technical components, and the processes by which these systems and components are created",
             "Stakholders affected by Benefit": ["one or more of AI User, AI Subject, Institutions, General Public and Environment"],
             "SDGs affected by Benefit": ["zero or more SDGs"],
             "Human Rights affected by Benefit": ["zero or more HR articles"]
            }},
            ],
      }},
      {{
        "Human Interaction": [
            {{"Benefit ID": "UseID-BID",
             "Benefit Description": "Benefit that targets the experience of people interacting with AI systems and their effects on these people",
             "Stakholders affected by Benefit": ["one or more of AI User, AI Subject, Institutions, General Public and Environment"],
             "SDGs affected by Benefit": ["zero or more SDGs"],
             "Human Rights affected by Benefit": ["zero or more HR articles"]
            }},
            ...
            {{"Benefit ID": "UseID-BID",
             "Benefit Description": "Benefit that targets the experience of people interacting with AI systems and their effects on these people",
             "Stakholders affected by Benefit": ["one or more of AI User, AI Subject, Institutions, General Public and Environment"],
             "SDGs affected by Benefit": ["zero or more SDGs"],
             "Human Rights affected by Benefit": ["zero or more HR articles"]
            }},
            ],
       }},
       {{
          "Systemic Impact": [
            {{"Benefit ID": "UseID-BID",
             "Benefit Description": "Benefit that the impact of an AI system has on the broader systems in which it is embedded, such as society, the economy, and the natural environment",
             "Stakholders affected by Benefit": ["one or more of AI User, AI Subject, Institutions, General Public and Environment"],
             "SDGs affected by Benefit": ["zero or more SDGs"],
             "Human Rights affected by Benefit": ["zero or more HR articles"]
            }},
            ...
            {{"Benefit ID": "UseID-BID",
             "Benefit Description": "Benefit that the impact of an AI system has on the broader systems in which it is embedded, such as society, the economy, and the natural environment",
             "Stakholders affected by Benefit": ["one or more of AI User, AI Subject, Institutions, General Public and Environment"],
             "SDGs affected by Benefit": ["zero or more SDGs"],
             "Human Rights affected by Benefit": ["zero or more HR articles"]
            }},
            ],
        }}
      ]
    }}

    ***Check your planned output before outputting it: if it contains any explanations besides the JSON string, omit the explanations. Make sure to output ONLY a correctly formatted JSON string and nothing else.***
    """
    }
]



def format_prompt(MESSAGES, useID, domain,purpose,aiCapability,aiUser,aiSubject,SDG_assessment,HR_assessment):
    S = "test {}"
    messages = deepcopy(MESSAGES)
    messages[1]['content'] = messages[1]['content'].format(useID, domain,purpose,aiCapability,aiUser,aiSubject,SDG_assessment,HR_assessment)
    return messages


In [20]:
# cost = 0

FULL_RES = []

start_time = time.time()
i = 0


for benefitID in benefit_ids:
  benefitID = str(benefitID)
  print (f" Parsing use {benefitID}")

  SDG_assessment = "[]"
  HR_assessment = "[]"

  for useSDG in SDG_benefits:
    useI = str(useSDG['id'])
    if useI == benefitID:
      SDG_assessment = str(useSDG["SDG Assessment"])
      break

  for useHR in HR_benefits:
    useI = str(useHR['id'])
    if useI == benefitID:
      HR_assessment = str(useHR["Assessment of impact on human rights"])
      break


  # Variables for message placeholders
  useID = benefitID
  domain = useSDG['Details'][0]
  purpose = useSDG['Details'][1]
  aiCapability = useSDG['Details'][2]
  aiUser = useSDG['Details'][3]
  aiSubject = useSDG['Details'][4]

  assert domain == useHR['Details'][0]
  assert aiSubject == useHR['Details'][4]

  # Extracting "Use i" details
  use_i_details = [domain,purpose,aiCapability,aiUser,aiSubject]

  print(use_i_details)

  # adapt the prompt for useI
  messages = format_prompt(MESSAGES, useID, domain,purpose,aiCapability,aiUser,aiSubject,SDG_assessment,HR_assessment)

  # run the prompt
  response = chat_gpt(messages, temperature=0)
  print(response)

  # response, token_count = get_completion_and_token_count(messages, temperature=0)
  # res = token_count
  # cost_chunk = (res['prompt_tokens'] * 0.03  + res['completion_tokens'] * 0.06)/1000.0
  # cost += cost_chunk

  response = ast.literal_eval(response)


  combined_response = {}
  combined_response["id"]= useI
  combined_response["Details"] = use_i_details
  combined_response["Benefits"] = response["Overall Benefit Assessment"]

  end_time = time.time()

  print(f"Execution time: {end_time - start_time:.5f} seconds")
  # print (f"TOTAL COST {cost}")

  FULL_RES.append(combined_response)

 Parsing use 54
['Entrepreneurship', 'Employee attendance tracking', 'Monitoring employee check-ins and check-outs', 'Business owners, HR managers', 'Employees']
{
  "Overall Benefit Assessment": [
    {
      "Capability": [
        {
          "Benefit ID": "54-1",
          "Benefit Description": "Monitors employee check-ins and check-outs, ensuring accurate attendance tracking.",
          "Stakeholders affected by Benefit": ["AI User", "AI Subject"],
          "SDGs affected by Benefit": ["SDG 8", "SDG 9"],
          "Human Rights affected by Benefit": ["Article 3", "Article 23", "Article 24"]
        }
      ]
    },
    {
      "Human Interaction": [
        {
          "Benefit ID": "54-2",
          "Benefit Description": "Promotes fair decision-making in promotions, potentially increasing diversity in leadership.",
          "Stakeholders affected by Benefit": ["AI User", "AI Subject"],
          "SDGs affected by Benefit": ["SDG 5"],
          "Human Rights affected by Benef

In [21]:
# FULL_RES

In [23]:
###############################
# save result
with open(f"../../results/benefit_categories/FULL_benefit_categorisation.json", "w") as json_file:
    json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation
# Download the file to your local machine

# THE END